In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertForSequenceClassification, BertModel
from sklearn.metrics import accuracy_score




In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def preprocess(text,max_seq_length):
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_seq_length,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    return inputs['input_ids'], inputs['attention_mask']

In [46]:
train_texts = ['This is a positive example.', 'This is a negative example.']
train_labels = [1, 0]

# for text in train_texts:
#     print(preprocess(text,128))

In [56]:
df = pd.read_csv('ClaimBuster_Datasets/datasets/groundtruth.csv')
df.head()

,Sentence_id,Text,Speaker,Speaker_title,Speaker_party,File_id,Length,Line_number,Sentiment,Verdict
0,26,"You know, I saw a movie - ""Crocodile Dundee.""",George Bush,Vice President,REPUBLICAN,1988-09-25.txt,9,26,0.000000,0
1,80,We're consuming 50 percent of the world's coca...,Michael Dukakis,Governor,DEMOCRAT,1988-09-25.txt,8,80,-0.740979,1
2,129,That answer was about as clear as Boston harbor.,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,9,129,0.000000,-1
3,131,Let me help the governor.,George Bush,Vice President,REPUBLICAN,1988-09-25.txt,5,131,0.212987,-1
4,172,We've run up more debt in the last eight years...,Michael Dukakis,Governor,DEMOCRAT,1988-09-25.txt,22,172,-0.268506,1


In [57]:
training_text = df['Text'].values
labels = df['Verdict'].values

In [55]:
training_text

array(['You know, I saw a movie - "Crocodile Dundee."',
       "We're consuming 50 percent of the world's cocaine.",
       'That answer was about as clear as Boston harbor.', ...,
       'Well, can I answer that?',
       'I look forward to the final weeks of this campaign.',
       'For those of you for me, thanks for your help.'], dtype=object)

In [68]:
train_input_ids = []
train_attention_masks = []
for text in training_text:
    inputs_id,input_attention = preprocess(text,128)
    train_input_ids.append(inputs_id)
    train_attention_masks.append(input_attention)
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)


In [69]:
model = BertModel.from_pretrained('bert-base-uncased', num_labels=3)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Train the model
batch_size = 2
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    optimizer.zero_grad()
    outputs = model(train_input_ids,train_attention_masks)
    outputs = torch.argmax(outputs,dim=1)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

train_loss /= len(train_labels) // batch_size

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:


# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the BERT model architecture
class BertClassifier(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        
        # self.fc2 = nn.Linear(hidden_size,hidden_size)
        self.fc = nn.Linear(hidden_size, num_labels)    

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        # logits = F.relu(self.fc2(pooled_output))
        logits = self.fc(logits)
        return logits

# Define the optimizer and loss function
model = BertClassifier(hidden_size=768, num_labels=2)
optimizer = optim.Adam(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Example training data
train_texts = ['This is a positive example.', 'This is a negative example.']
train_labels = [1, 0]

# Tokenize the texts and convert to input features
train_input_ids = []
train_attention_masks = []
for text in train_texts:
    inputs_id,input_attention = preprocess(text,128)
    train_input_ids.append(inputs_id)
    train_attention_masks.append(input_attention)
train_input_ids = torch.cat(train_input_ids, dim=0)
train_attention_masks = torch.cat(train_attention_masks, dim=0)
train_labels = torch.tensor(train_labels)





Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
outputs = model(train_input_ids,train_attention_masks)

